In [ ]:
class Leaf:
    def __init__(self,origin,left,right=''):
        self.origin = origin
        if len(left) == 0:
            self.left = '$'
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(None,right)
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)+len(self.right)
    
    # def __str_(self): return '\t' + str(self.left) + ':' + str(self.right) + '\n'

In [ ]:
class Branch:
    def __init__(self):
        self.b = {}

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index): return self.b[index]
    
    def __contains__(self,other): return other in self.b

    def __len__(self): return len(self.b)

    def add(self,leaf):
        if leaf in self.b:
            if self.b[leaf].left == '$':
                self.b[leaf].right+=1
                return
            i = 0
            left = ''
            while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
                left += leaf.left[i]
                i += 1
            if type(self.b[leaf].right) == Branch:
                nleaf = Leaf(leaf.origin,leaf.left[i:])
                if len(self.b[leaf].left[i:]):
                    branch = Branch()
                    l = Leaf(self.b[leaf].origin,self.b[leaf].left[i:])
                    l.right = self.b[leaf].right
                    branch.b[l] = l
                    self.b[leaf].right = branch
                self.b[leaf].right.add(nleaf)
            else: 
                branch = Branch()
                branch.add(Leaf(self.b[leaf].origin,self.b[leaf].left[i:]))
                branch.add(Leaf(leaf.origin,leaf.left[i:]))
                self.b[leaf].right = branch
            self.b[leaf].left = left
        else: 
            self.b[leaf] = leaf

In [ ]:
class Sequence:
    def __init__(self):
        self.reads = []
        self.suffix_options = []
        self.prefix_options = []

In [ ]:
class Sequitur:
    def __init__(self):
        self.reads = []
        self.sequence = ''

In [ ]:
b = Branch()
texts = ['hello','hollow','bellow']
for text in texts:
    for i in range(len(text)): b.add(Leaf(text,text[i:]))
print(b)

In [ ]:
b = Branch()
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']
for read in reads:
    for i in range(len(read)): b.add(Leaf(read,read[i:]))
print(b)

In [ ]:
# TS | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | betty_bought_butter_th
print('h:\t',b.b['h'])
print('th:\t',b.b['t'].right['h'])
print('_th:\t',b.b['_'].right['t'].right['h'])
print('r_th:\t',b.b['r'].right['_'].right['t'].right['h'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['r'].right['_'].right['t'].right['h'].right.b:
    print(a.origin) 
    # betty_bought_butter_th
    # betty_bought_butter_th: FALSE [IDENTICAL]
    
    # betty_bought_butter_th
    #                tter_the_butter_was_: TRUE

print()
print("TRUE SUCCESS")

In [ ]:
# TS | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | tter_the_butter_was_
print('_:\t',b.b['_'])
print('s_:\t',b.b['s'])
print('as_:\t',b.b['a'].right['s'])
print('was_:\t',b.b['w'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['w'].right.b:
    print(a.origin) 
    # tter_the_butter_was_
    # tter_the_butter_was_: FALSE [IDENTICAL]

    #       he_butter_was_bitter_
    # tter_the_butter_was_: TRUE

print()
print("TRUE SUCCESS")

In [ ]:
# TS | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | he_butter_was_bitter_
print('_:\t',b.b['_'])
print('r_:\t',b.b['r'].right['_'])
print('er_:\t',b.b['r'].right['_'])
print('ter_:\t',b.b['t'].right['e'].right['_'])
print('tter_:\t',b.b['t'].right['t'])
print('itter_:\t',b.b['i'])
print('bitter_:\t',b.b['b'].right['i'])
print('_bitter_:\t',b.b['_'].right['b'].right['i'])
print('s_bitter_:\t',b.b['s'].right['b'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['s'].right['b'].right.b:
    print(a.origin) 
    # he_butter_was_bitter_
    # he_butter_was_bitter_: FALSE [IDENTICAL]

    #            as_bitter_betty_bought
    # he_butter_was_bitter_: TRUE

print()
print("TRUE SUCCESS")

In [ ]:
# FF | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | as_bitter_betty_bought
print('t:\t',b.b['t'])
print('ht:\t',b.b['h'].right['t'])
print('ght:\t',b.b['g'])
print('ught:\t',b.b['u'].right['g'])
print('ought:\t',b.b['o'].right['u'])
print('bought:\t',b.b['b'].right['o'])
print('_bought:\t',b.b['_'].right['b'].right['o'])
print('y_bought:\t',b.b['y'].right['_'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['y'].right['_'].right.b:
    print(a.origin) 
    #           betty_bought_butter_th
    # as_bitter_betty_bought: TRUE *but shouldn't be* alternate condition: 2 ENTRIES WHERE ONE IS AN END NODE?

    #             tty_bought_better_butter_t
    # as_bitter_betty_bought: TRUE

print()
print("FALSE FAILURE")

# TIE BREAKER? [FORWARD TRAVERSAL]
# Sequitur class
#   # pulls reads randomly out of queue
#   # attempts to extend
#   # if extension is unsuccessful because there are several options, extend the options to eliminate some

In [ ]:
# TS | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | tty_bought_better_butter_t
print('t:\t',b.b['t'])
print('_t:\t',b.b['_'].right['t'])
print('r_t:\t',b.b['r'].right['_'].right['t'])
print('er_t:\t',b.b['e'].right['r'].right['_'].right['t'])
print('ter_t:\t',b.b['t'].right['e'].right['_'].right['t'])
print('tter_t:\t',b.b['t'].right['t'].right['e'].right['_'].right['t'])
print('utter_t:\t',b.b['u'].right['t'].right['t'])
print('butter_t:\t',b.b['b'].right['u'].right['t'])
print('_butter_t:\t',b.b['_'].right['b'].right['u'].right['t'])
print('r_butter_t:\t',b.b['r'].right['_'].right['b'].right['u'].right['t'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['r'].right['_'].right['b'].right['u'].right['t'].right.b:
    print(a.origin) 
    # tty_bought_better_butter_t
    # tty_bought_better_butter_t: FALSE [IDENTICAL]

    #                 r_butter_to_make_the_
    # tty_bought_better_butter_t: TRUE

print()
print("TRUE SUCCESS")

In [ ]:
# TS | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | r_butter_to_make_the_
print('_:\t',b.b['_'])
print('e_:\t',b.b['e'].right['_'])
print('he_:\t',b.b['h'].right['e'])
print('the_:\t',b.b['t'].right['h'].right['e'])
print('_the_:\t',b.b['_'].right['t'].right['h'].right['e'])
print('e_the_:\t',b.b['e'].right['_'].right['t'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX AND NOT IDENTICAL TO THE SUFFIX]
for a in b.b['e'].right['_'].right['t'].right.b:
    print(a.origin) 
    # r_butter_to_make_the_
    # r_butter_to_make_the_: FALSE [IDENTICAL]

    #               ke_the_bitter_butter_better
    # r_butter_to_make_the_: TRUE

print()
print("TRUE SUCCESS")

In [ ]:
# TF | TRAVERSAL [UNTIL THE BRANCH HAS 2 ENTRIES] | ke_the_bitter_butter_better
print('r:\t',b.b['r'])
print('er:\t',b.b['e'].right['r'])
print('tter:\t',b.b['t'].right['t'].right['e'])
print('etter:\t',b.b['e'].right['t'].right['e'])
print()

# CONFIRMATION [IS THE RESULT A PREFIX OF THE SUFFIX]: FALSE
for a in b.b['e'].right['t'].right['e'].right.b:
    print(a.origin) # tty_bought_better_butter_t
    #ke_the_bitter_butter_better
    #ke_the_bitter_butter_better: FALSE [IDENTICAL]

    #          tty_bought_better_butter_t
    #ke_the_bitter_butter_better: FALSE [MISMATCH]

print()
print("TRUE FAILURE")